# Probar Web API ya creada en forma pública para utilizar modelo entrenado con TensorFlow+Keras usando Flash y ngrok

In [6]:
#@title Configuración

ngrok_public_Web_API = "https://c772-34-68-28-183.ngrok-free.app/v1/models/IRIS:predict" #@param{type:"string"}



In [1]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
#@title Cargar complementos del Modelo ya entrenado
import os
import joblib

path_modelo = '/content/gdrive/My Drive/IA/demoModelDeployment/modelo'  #@param {type:"string"}


# cargar scaler (si existe)
fn_scaler = path_modelo+"/scaler.joblib"
if os.path.isfile(fn_scaler):
  scaler = joblib.load(fn_scaler)
  print("\n* Scaler cargado de ", fn_scaler, "\n")
else:
  scaler = None
  print("\n* Scaler no encontrado en ", fn_scaler, "\n")

fn_clases = path_modelo+"/CLASES.txt"
CLASES = []
if os.path.isfile(fn_clases):
  with open(fn_clases, 'r') as f:
    # carga datos
    auxData = f.readlines()
  for c in auxData:
    CLASES.append( c.replace("\n", "") )
  print("\n* CLASES definidas cargado de ", fn_clases, ":")
  print("\t\t", CLASES, "\n")
else:
  print("\n* CLASES no encontradas en ", fn_clases, "\n")



* Scaler no encontrado en  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/scaler.joblib 


* CLASES definidas cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/CLASES.txt :
		 ['na', 'Setosa', 'Versicolor', 'Virginica'] 



In [7]:
#@title Probar Modelo usando TensorFlow Serving público

import numpy as np
import requests
import json

def ejecutarModeloURL(vals, modelURL):
  headers = {"content-type": "application/json", "ngrok-skip-browser-warning": "69420" }
  data_json = json.dumps({"signature_name": "serving_default", "instances": vals})
  json_response = requests.post(modelURL, data=data_json, headers=headers)
  if json_response.status_code == 200:
    return json.loads(json_response.text)
  else:
    print(json_response, ":", json_response.content)
    return "ERROR"

def clasificarIris(valMedidas, modelURL):
  # normaliza los datos (si el scaler está definido)
  if (scaler is not None):
    valMedidas = scaler.transform(valMedidas)
  # ejecuta el modelo
  resModel = ejecutarModeloURL(valMedidas, modelURL)
  # se genera al menos un resultado
  if (len(resModel) > 0) and ("predictions" in resModel):
    resClases = []
    for r in resModel["predictions"]:
      if (len(r) == 1):
        # como tiene una salida se asume salida lineal (solo la redondea)
        claseID = round(r)
      else:
        # como tiene validas salidas se asume salida softmax (toma la de mayor puntaje)
        claseID = int( np.argmax(r, axis=0) )
      # determina la descripción de la clase (si está definida)
      if (CLASES is not None) and (len(CLASES)>0) and (claseID<len(CLASES)):
        claseDesc = CLASES[claseID]
      else:
        claseDesc = str(claseID)
      resClases.append( { "claseID" : claseID, "clase" : claseDesc} )
    return resClases
  else:
    print(resModel)
    return []

# ejecuta
LargoSepalo = 5.5 #@param{type:"number"}
AnchoSepalo = 2.6 #@param{type:"number"}
LargoPetalo = 4.4 #@param{type:"number"}
AnchoPetalo = 1.2 #@param{type:"number"}

vals = [[LargoSepalo, AnchoSepalo, LargoPetalo, AnchoPetalo]]
res = clasificarIris(vals, ngrok_public_Web_API)
if len(res)==0:
  print("--> Clase no definida!")
else:
  for r in res:
    print("--> Clase " + str(r))

--> Clase {'claseID': 2, 'clase': 'Versicolor'}


In [8]:
#@title Probar Modelo usando TensorFlow Serving público con ejemplos de un CSV que tiene los datos mostrados en <construir-RNA-MLP-IRIS.ipynb>
from google.colab import files

def cargarArchivo(fn):

  # lo carga en lista
  uploadedData = []
  with open(fn, 'r', encoding='utf-8') as f:
    contents = f.readlines()
  # separa en lineas
  uploadedData.extend( ("\n".join(contents)).split("\n") )

  print('-- Archivo "{name}" con largo {length} bytes cargado'.format(
    name=fn, length=len(uploaded[fn])))

  return uploadedData

# sube archivo
uploaded = files.upload()
# procesa los datos
uploadedData = []
for fn in uploaded.keys():
    uploadedData.extend( cargarArchivo(fn))

if len(uploadedData)>0:
  print("\n")
  # procesa los datos cargados
  classReal = []
  classPreds = []
  for data in uploadedData:
    if data!="":
      arAux = data.split(";")
      vals = [[float(arAux[0]), float(arAux[1]), float(arAux[2]), float(arAux[3])]]
      classReal.append( arAux[4] )
      claseModelo = clasificarIris(vals, ngrok_public_Web_API)
      if len(claseModelo)==0:
        classPreds.append("Clase no definida!")
      else:
        classPreds.append( str(claseModelo[0]["claseID"]) )
  print("\n")


  from sklearn.metrics import classification_report
  from sklearn.metrics import confusion_matrix
  import pandas as pd

  # muestra reporte de clasificación
  print("\n Reporte de Clasificación: ")
  print(classification_report(classReal, classPreds))

  # muestra matriz de confusion
  print('\nMatriz de Confusión ( real / modelo ): ')
  cm = confusion_matrix(classReal, classPreds)
  cmtx = pd.DataFrame(
      cm
    )
  # agrega para poder mostrar la matrix de confusión completa
  pd.options.display.max_rows = 100
  pd.options.display.max_columns = 100
  cmtx.sort_index(axis=0, inplace=True)
  cmtx.sort_index(axis=1, inplace=True)
  print(cmtx)
  print("\n")


Saving datos_PRUEBA.csv to datos_PRUEBA.csv
-- Archivo "datos_PRUEBA.csv" con largo 684 bytes cargado





 Reporte de Clasificación: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       0.92      0.85      0.88        13
           3       0.86      0.92      0.89        13

    accuracy                           0.92        38
   macro avg       0.92      0.92      0.92        38
weighted avg       0.92      0.92      0.92        38


Matriz de Confusión ( real / modelo ): 
    0   1   2
0  12   0   0
1   0  11   2
2   0   1  12


